In [ ]:
# importing pytorch
import torch
print(torch.__version__)


2.5.1.post108
CUDA available: False


In [ ]:
# running data config file
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate/model_boilerplate_remote.py'

In [ ]:
# loading in uncentered-data-trained RNN
loaded_model = torch.load('/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt')

/tmp/ipykernel_1866008/1907153318.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load('/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/s

In [4]:
# getting predicted values y_pred from test set 
rnn_y_pred = loaded_model.predict(x_test) 
print(type(rnn_y_pred), rnn_y_pred.shape)

AttributeError: 'dict' object has no attribute 'predict'